In [28]:
# 查看数据文件目录  list datalab files
!ls datalab/

231784	60822


In [29]:
# 查看个人永久空间文件  list files in your permanent storage
!ls /home/tianchi/myspace/


data  model  result


In [ ]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

In [ ]:
# 绘图案例 an example of matplotlib
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jn
from IPython.display import display, clear_output
import time
x = np.linspace(0,5)
f, ax = plt.subplots()
ax.set_title("Bessel functions")

for n in range(1,10):
    time.sleep(1)
    ax.plot(x, jn(x,n))
    clear_output(wait=True)
    display(f)

# close the figure at the end, so we don't get a duplicate
# of the last plot
plt.close()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
# 排除警告信息
import warnings
# matplotlib画图常见参数设置
mpl.rcParams["font.family"] = "SimHei" 
# 设置字体
mpl.rcParams["axes.unicode_minus"]=False 
# 用来正常显示负号
plt.rcParams['font.sans-serif']=['SimHei'] 
# 用来正常显示中文标签# 嵌入式显示图形
%matplotlib inline
warnings.filterwarnings("ignore")

In [3]:
path='datalab/60822/'
file_name='train_test_data.csv'
raw_data=pd.read_csv(path+file_name)
raw_data.shape

(200000, 112)

In [4]:
#数据分隔，这是上次特征工程后的数据，将其分隔为训练集与测试集，按索引划分
train=raw_data.loc[0:149999]
test=raw_data.loc[150000:200000]
train.columns

Index(['train', 'power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5',
       'v_6',
       ...
       'fuelType_5.0', 'fuelType_6.0', 'gearbox_0.0.1', 'gearbox_1.0.1',
       'notRepairedDamage_-.1', 'notRepairedDamage_0.0.1',
       'notRepairedDamage_1.0.1', 'model_class_0', 'model_class_1',
       'model_class_2'],
      dtype='object', length=112)

In [5]:
drop_list=['train', 'name', 'regionCode', 'is_fuel_0', 'is_fuel_1', 'train.1',
           'is_low_seasons_0', 'is_low_seasons_1', 'name.1', 'regionCode.1', 'bodyType_0.0',
           'bodyType_2.0', 'bodyType_3.0', 'bodyType_7.0', 'fuelType_2.0', 'fuelType_3.0', 'fuelType_4.0',
           'fuelType_5.0', 'fuelType_6.0', 'model_class_1', 'model_class_2']
len(drop_list)

21

In [6]:
train.drop(drop_list,axis=1,inplace=True)
test.drop(drop_list,axis=1,inplace=True)
target=pd.read_csv('datalab/231784/used_car_train_20200313.csv',sep=' ')
target=target['price']
target.shape

(150000,)

In [7]:
test.fillna(test.median(),inplace=True)
train.fillna(train.median(),inplace=True)#处理缺失值，中位数填充
train.isnull().sum().sort_values(ascending=False)/len(train)

model_class_0              0.0
v_std                      0.0
brand                      0.0
gearbox_count              0.0
gearbox_price_average      0.0
gearbox_price_max          0.0
gearbox_price_median       0.0
gearbox_price_min          0.0
gearbox_price_sum          0.0
gearbox_std                0.0
brand_count                0.0
brand_price_average        0.0
brand_price_max            0.0
brand_price_median         0.0
brand_price_min            0.0
brand_price_sum            0.0
brand_std                  0.0
model_count                0.0
model_price_average        0.0
model_price_max            0.0
model_price_median         0.0
model                      0.0
v_mean                     0.0
model_price_sum            0.0
v_sum                      0.0
kilometer                  0.0
v_0                        0.0
v_1                        0.0
v_2                        0.0
v_3                        0.0
                          ... 
fuelType_0.0               0.0
fuelType

In [8]:
#设置随机数种子，保证结果的可复现性
np.random.seed(123)
from keras.models import Sequential # Keras model module
from keras.layers.core import Dense, Dropout, Activation

In [9]:
#划分数据集,对数据集进行归一化,对目标值进行取对数处理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_data=sc.fit_transform(train.values)
val_data=sc.fit_transform(test.values)
y=target.values
train_x,test_x,train_y,test_y=train_test_split(train_data,y,test_size=0.2)

In [22]:
import keras
from keras.optimizers import SGD,adam
from sklearn.metrics import mean_absolute_error
from keras.layers import Conv1D, Activation, MaxPool1D, Flatten, Dense
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, merge, Add
from keras.optimizers import Adam



init = keras.initializers.glorot_uniform()
model = keras.models.Sequential()
model.add(Dense(units=300, input_shape=(91,), kernel_initializer=init, activation='softplus'))
#model.add(Dropout(0.2))
model.add(Dense(units=300, kernel_initializer=init, activation='softplus'))
#model.add(Dropout(0.2))
model.add(Dense(units=64, kernel_initializer=init, activation='softplus'))
model.add(Dense(units=32, kernel_initializer=init, activation='softplus'))
model.add(Dense(units=8, kernel_initializer=init, activation='softplus'))
model.add(Dense(units=1))
model.summary()

simple_adam = Adam(lr = 0.015)
model.compile(loss = 'mse',optimizer=simple_adam)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 300)               27600     
_________________________________________________________________
dense_20 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_21 (Dense)             (None, 64)                19264     
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 9         
Total params: 139,517
Trainable params: 139,517
Non-trainable params: 0
_________________________________________________________________


In [23]:
epoch=20
model.fit(train_x,train_y,nb_epoch = epoch,batch_size = 6,validation_data = (test_x,test_y))
model.save_weights(path+'modelfile.model')
y_pred=model.predict(test_x) 
# 模型评估
print('The rmse of prediction is:', mean_absolute_error(test_y, y_pred))

Train on 120000 samples, validate on 30000 samples
Epoch 1/20
120000/120000 [==============================] - 54s 452us/step - loss: 7163328.0653 - val_loss: 8396767.2744
Epoch 2/20
120000/120000 [==============================] - 55s 455us/step - loss: 5234968.0700 - val_loss: 3219598.7916
Epoch 3/20
120000/120000 [==============================] - 52s 437us/step - loss: 4840308.2483 - val_loss: 4515484.3617
Epoch 4/20
120000/120000 [==============================] - 54s 449us/step - loss: 4544334.7311 - val_loss: 7606591.6321
Epoch 5/20
120000/120000 [==============================] - 51s 429us/step - loss: 4328044.1939 - val_loss: 6723241.6518
Epoch 6/20
120000/120000 [==============================] - 51s 422us/step - loss: 4377011.9863 - val_loss: 2865079.9686
Epoch 7/20
120000/120000 [==============================] - 50s 421us/step - loss: 3742974.1028 - val_loss: 2560671.4401
Epoch 8/20
120000/120000 [==============================] - 53s 441us/step - loss: 3590033.7901 - val_

In [10]:
#其他人的训练方法
from keras.layers import Conv1D, Activation, MaxPool1D, Flatten, Dense
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, merge, Add
def NN_model(input_dim):
    init = keras.initializers.glorot_uniform(seed=1)
    model = keras.models.Sequential()
    model.add(Dense(units=300, input_dim=input_dim, kernel_initializer=init, activation='softplus'))
    #model.add(Dropout(0.2))
    model.add(Dense(units=300, kernel_initializer=init, activation='softplus'))
    #model.add(Dropout(0.2))
    model.add(Dense(units=64, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=32, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=8, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=1))
    return model


In [11]:
from keras.callbacks import Callback, EarlyStopping
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred3 = self.model.predict(X_train)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = y_pred3[i]
        for i in range(len(y_pred3)):
            y_true[i] = y_train[i]
        trn_s = mean_absolute_error(y_true, y_pred)
        logs['trn_score'] = trn_s
        
        X_val, y_val = self.data[1][0], self.data[1][1]
        y_pred3 = self.model.predict(X_val)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = y_pred3[i]
        for i in range(len(y_pred3)):
            y_true[i] = y_val[i]
        val_s = mean_absolute_error(y_true, y_pred)
        logs['val_score'] = val_s
        print('trn_score', trn_s, 'val_score', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [12]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler
  
def scheduler(epoch):
    # 每隔100个epoch，学习率减小为原来的1/10
    if epoch % 20 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.6)
        print("lr changed to {}".format(lr * 0.6))
    return K.get_value(model.optimizer.lr)
reduce_lr = LearningRateScheduler(scheduler)
#model.fit(train_x, train_y, batch_size=32, epochs=5, callbacks=[reduce_lr])

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import keras 

n_splits = 6
kf = KFold(n_splits=n_splits, shuffle=True)
b_size = 2000
max_epochs = 145
oof_pred = np.zeros((len(train_data), ))

sub = pd.read_csv('datalab/231784/used_car_testA_20200313.csv',sep = ' ')[['SaleID']].copy()
sub['price'] = 0

avg_mae = 0
for fold, (trn_idx, val_idx) in enumerate(kf.split(train_data, y)):
    print('fold:', fold)
    X_train, y_train = train_data[trn_idx], y[trn_idx]
    X_val, y_val = train_data[val_idx], y[val_idx]
    
    model = NN_model(X_train.shape[1])
    simple_adam = keras.optimizers.Adam(lr = 0.015)
    
    model.compile(loss='mae', optimizer=simple_adam,metrics=['mae'])
    es = EarlyStopping(monitor='val_score', patience=10, verbose=2, mode='min')
    es.set_model(model)
    metric = Metric(model, [es], [(X_train, y_train), (X_val, y_val)])
    model.fit(X_train, y_train, batch_size=b_size, epochs=max_epochs, 
              validation_data = [X_val, y_val],
              callbacks=[reduce_lr], shuffle=True, verbose=2)
    y_pred3 = model.predict(X_val)
    y_pred = np.zeros((len(y_pred3), ))
    sub['price'] += model.predict(test).reshape(-1,)/n_splits
    for i in range(len(y_pred3)):
        y_pred[i] = y_pred3[i]
        
    oof_pred[val_idx] = y_pred
    val_mae = mean_absolute_error(y[val_idx], y_pred)
    avg_mae += val_mae/n_splits
    print()
    print('val_mae is:{}'.format(val_mae))
    print()



fold: 0
Train on 125000 samples, validate on 25000 samples
Epoch 1/145
 - 4s - loss: 1962.3983 - mean_absolute_error: 1962.3983 - val_loss: 1182.1167 - val_mean_absolute_error: 1182.1167
Epoch 2/145
 - 3s - loss: 1080.1999 - mean_absolute_error: 1080.1999 - val_loss: 1016.0823 - val_mean_absolute_error: 1016.0823
Epoch 3/145
 - 3s - loss: 945.2840 - mean_absolute_error: 945.2840 - val_loss: 1096.6383 - val_mean_absolute_error: 1096.6383
Epoch 4/145
 - 4s - loss: 834.0246 - mean_absolute_error: 834.0246 - val_loss: 798.7523 - val_mean_absolute_error: 798.7523
Epoch 5/145
 - 3s - loss: 741.5724 - mean_absolute_error: 741.5724 - val_loss: 727.1586 - val_mean_absolute_error: 727.1586
Epoch 6/145
 - 3s - loss: 751.2554 - mean_absolute_error: 751.2554 - val_loss: 788.0444 - val_mean_absolute_error: 788.0444
Epoch 7/145
 - 3s - loss: 703.1364 - mean_absolute_error: 703.1364 - val_loss: 689.7323 - val_mean_absolute_error: 689.7323
Epoch 8/145
 - 3s - loss: 653.1681 - mean_absolute_error: 653.1

NameError: name 'mean_absolute_error' is not defined

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y, oof_pred)
sub.to_csv('nn_sub_{}_{}.csv'.format('mae', sub['price'].mean()), index=False)

In [17]:
print(mean_absolute_error(y, oof_pred))

5007.6054862


In [18]:
y_pre=model.predict(train_data)
print(mean_absolute_error(y, y_pre))

402.948189403


In [19]:
test_price=model.predict(val_data)
test_price

array([[ 35113.6328125 ],
       [   319.2571106 ],
       [  6171.37646484],
       ..., 
       [  7525.73388672],
       [ 10053.48828125],
       [  3503.63061523]], dtype=float32)

In [31]:
sub['test_price_pr']=test_price
sub.to_csv('myspace/result/test_pridict.csv', index=False)